# Rock, Paper, Scissors

## Imports

In [181]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import TensorBoard
from keras.utils import to_categorical
import datetime

## Verbose
If this flag is set to true, cells will give more detailed output

In [182]:
verbose = False

## Read csv

In [183]:
data = pd.read_csv('/home/carol/Documents/Master/3_Semester/Anwendung der KI/Project/data.csv')
if verbose:
    print(data.head())

## Transforming the data into numerical values

In [184]:
# P = 0; R = 1; S = 2
label_encoder = LabelEncoder()
for column in data.columns:
    data[column] = label_encoder.fit_transform(data[column])

if verbose: 
    print(data.head())  


## Making input and Output 
- The input data is an array with the picks of the player and the computer in the last n rounds. The amount of rounds if the defined in the variable sequence_length
- The output data, what the computer is trying to guess, is the next human pick.

In [185]:
sequence_length = 3
X = []
y = []

for i in range(len(data) - sequence_length):
    X.append(data[['H1', 'C1']].iloc[i:i+sequence_length].values)
    y.append(data['H1'].iloc[i + sequence_length])

X = np.array(X)
y = np.array(y)

if verbose:
    print(X[0])
    print(y[0])
    print(X[1])
    print(y[1])
    print(X[2])
    print(y[2])


## Spliting the data into train and test set

In [186]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Changing the output data to hot encoding

In [187]:
output_dim = len(label_encoder.classes_)
y_train = to_categorical(y_train, num_classes=output_dim)
y_test = to_categorical(y_test, num_classes=output_dim)

if verbose:
    print(y_train[0])

## Tune the Model

In [188]:
hiddenUnits = 10
stackLSTM = False # The model will include Dropout and hiddenUnits layers if True
dropout = 0.1
input_dim = X_train.shape[2] # Human and Computer
timestep_length = X_train.shape[1] # 5 last rounds
if not verbose: 
    print(f'input_dim: {input_dim}')
    print(f'timestep_length: {timestep_length}')

input_dim: 2
timestep_length: 3


In [189]:
model = Sequential()
model.add(LSTM(hiddenUnits, return_sequences=stackLSTM, input_shape=(timestep_length, input_dim)))
if stackLSTM:
    model.add(Dropout(dropout))
    model.add(LSTM(hiddenUnits))
model.add(Dropout(dropout))
model.add(Dense(output_dim, activation='softmax'))

/home/carol/Documents/Master/3_Semester/Anwendung der KI/Project/.env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Train

In [190]:
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tbCallBack = TensorBoard(log_dir='./logs/' + now, histogram_freq=1, write_graph=True, write_images=False)

showProgress = 0
if verbose: 
    showProgress = 2

model.compile(loss='categorical_crossentropy')
model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    verbose=showProgress,
    validation_split=0.2,
    callbacks=[tbCallBack]
)


## Evaluate the Model

In [ ]:
loss = model.evaluate(X_test, y_test)
print(loss)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 1.1091
1.1091113090515137


## Predict the Next Move

In [192]:
new_moves = np.array([[[label_encoder.transform(['R'])[0], label_encoder.transform(['P'])[0]],
                       [label_encoder.transform(['P'])[0], label_encoder.transform(['R'])[0]],
                       [label_encoder.transform(['S'])[0], label_encoder.transform(['S'])[0]]]])  # Replace with actual new moves
predicted_move = model.predict(new_moves)
predicted_move = label_encoder.inverse_transform([np.argmax(predicted_move)])
print(f'Predicted next move: {predicted_move[0]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Predicted next move: S


## Determine the counter move for the computer

### Function Implementation

In [193]:
def counter_move(player_move):
    if player_move == 'R':
        return 'P'  # Paper beats Rock
    elif player_move == 'P':
        return 'S'  # Scissors beats Paper
    elif player_move == 'S':
        return 'R'  # Rock beats Scissors

### Function Call

In [194]:
computer_move = counter_move(predicted_move[0])
print(f'Computer should play: {computer_move}')

Computer should play: R
